In [1]:
# !pipenv install requests-html requests

In [2]:
import requests
import time
import pandas as pd
from requests_html import HTML

In [3]:
base_url = 'https://stackoverflow.com/questions/tagged/'
tag = "python"
query_filter = "Votes"
url = f"{base_url}{tag}?tab={query_filter}"
url

'https://stackoverflow.com/questions/tagged/python?tab=Votes'

In [4]:
r = requests.get(url)
html_str = r.text
html = HTML(html = html_str)

In [5]:
question_classes = html.find(".s-link")       # selenium
question_summaries = html.find(".s-post-summary")
print (question_summaries)

[<Element 'div' id='question-summary-231767' class=('s-post-summary', 'js-post-summary') data-post-id='231767' data-post-type-id='1'>, <Element 'div' id='question-summary-419163' class=('s-post-summary', 'js-post-summary') data-post-id='419163' data-post-type-id='1'>, <Element 'div' id='question-summary-394809' class=('s-post-summary', 'js-post-summary') data-post-id='394809' data-post-type-id='1'>, <Element 'div' id='question-summary-100003' class=('s-post-summary', 'js-post-summary') data-post-id='100003' data-post-type-id='1'>, <Element 'div' id='question-summary-82831' class=('s-post-summary', 'js-post-summary') data-post-id='82831' data-post-type-id='1'>, <Element 'div' id='question-summary-38987' class=('s-post-summary', 'js-post-summary') data-post-id='38987' data-post-type-id='1'>, <Element 'div' id='question-summary-89228' class=('s-post-summary', 'js-post-summary') data-post-id='89228' data-post-type-id='1'>, <Element 'div' id='question-summary-273192' class=('s-post-summary'

In [6]:
print(question_summaries[0].text)

12217 votes
49 answers
3.0m views
What does the "yield" keyword do?
What is the use of the yield keyword in Python? What does it do? For example, I'm trying to understand this code1: def _get_child_candidates(self, distance, min_dist, max_dist): if self._leftchild ...
python iterator generator
Alex. S.
138k
asked Oct 23, 2008 at 22:21


In [7]:
columns = ['votes', 'num_answers', 'views', 'question', 'short_desc', 'tags', 'user', 'user_detail', 'date']

In [8]:
#this_row = list(question_summaries[0].text.split("\n"))
#this_row

In [9]:
#len(this_row) == len (columns)

In [10]:
#row_data = dict(zip(columns, this_row))
#row_data

In [11]:
# nahrada
#for column, row_v in zip(columns, this_row):
#    print(column, row_v)

In [12]:
key_names = ['question', 'votes', 'tags', 'qu_id']
classes_needed = ['.s-link', '.s-post-summary--stats-item__emphasized', '.tags','div.data-post-id']
this_question_element = question_summaries[0]
this_question_element.find('.s-link', first=True).text
this_question_element.find('.s-post-summary--stats-item__emphasized', first=True).text.replace(' votes', '')

'12217'

In [13]:
def clean_scraped_data(text, keyname = None):
    if keyname == 'votes':
        return text.replace(' votes', '')
    return text

In [16]:
datas = []

for q_el in question_summaries:
    question_data = {}
    for i, _clas in enumerate(classes_needed):
        sub_el = q_el.find(_clas, first=True)
        key_name = key_names[i]
        if i == 3:
            question_data[key_name] = q_el.attrs['data-post-id']
            print (key_name, question_data[key_name])
        else:
            question_data[key_name] = clean_scraped_data(sub_el.text, keyname = key_name)
    datas.append(question_data)


qu_id 231767
qu_id 419163
qu_id 394809
qu_id 100003
qu_id 82831
qu_id 38987
qu_id 89228
qu_id 273192
qu_id 522563
qu_id 952914
qu_id 136097
qu_id 509211
qu_id 176918
qu_id 3294889
qu_id 423379
qu_id 3437059
qu_id 415511
qu_id 6470428
qu_id 16476924
qu_id 3207219
qu_id 1436703
qu_id 606191
qu_id 613183
qu_id 123198
qu_id 1024847
qu_id 448271
qu_id 1720421
qu_id 53513
qu_id 1132941
qu_id 986006
qu_id 2612802
qu_id 17071871
qu_id 252703
qu_id 510348
qu_id 36901
qu_id 739654
qu_id 576169
qu_id 6996603
qu_id 332289
qu_id 2052390
qu_id 312443
qu_id 287871
qu_id 4906977
qu_id 466345
qu_id 30081275
qu_id 5137497
qu_id 1602934
qu_id 11346283
qu_id 379906
qu_id 11277432


In [ ]:
#datas

In [ ]:
def parse_tagged_page(html):
    question_summaries = html.find(".s-post-summary")
    key_names = ['question', 'votes', 'tags', 'question_id']
    classes_needed = ['.s-link', '.s-post-summary--stats-item__emphasized', '.tags', '.s-link']
    datas = []
    for q_el in question_summaries:
        question_data = {}
        for i, _clas in enumerate(classes_needed):
            sub_el = q_el.find(_clas, first=True)
            key_name = key_names[i]
            if i == 3:
                question_data[key_name] = q_el.attrs['data-post-id']
            question_data[key_name] = clean_scraped_data(sub_el.text, keyname = key_name)
        datas.append(question_data)
    return datas

In [ ]:
def extract_data_from_url(url):
    r = requests.get(url)
    if r.status_code not in range(200, 299):
        return []
    html_str = r.text
    html = HTML(html = html_str)
    datas = parse_tagged_page(html)
    return datas

In [ ]:
def scrape_tag(tag="python", query_filter = "Votes", page=1, max_pages=50, pagesize = 25):
    base_url = 'https://stackoverflow.com/questions/tagged/'
    datas = []
    for p in range (max_pages):
        page_num = p+1
        url = f"{base_url}{tag}?tab={query_filter}&page={page_num}&pagesize={pagesize}"
        datas += extract_data_from_url(url)
        time.sleep(1.2)
    return datas


In [ ]:
datas = scrape_tag(tag = 'python', max_pages=5)
len(datas)

In [ ]:
df = pd.DataFrame(datas)
df.head()

In [ ]:
df.shape

In [ ]:
df.to_csv("python.csv", index = False)

In [ ]:
# To-do after machine learning : auto-tagging question bot

In [ ]:
url = 'https://stackoverflow.com/questions/231767/'
regex_pattern = r"https://stackoverflow.com/questions/(P<product_id>[0-9]+)/"
regex_options = [
    r"https://stackoverflow.com/questions/(P<question_id>[0-9]+)/",
    r"https://gude.gude.sk/(?P<slug>[\w-]+)/(?P<question_id>[0-9]+).html"
]
def extract_question_id_from_url(url):
    question_id = None
    for regex_str in regex_options:
        regex = re.compile(regex_str)
        match = regex.match(url)
        if match != None:
            try:
                question_id = match['question_id']
            except:
                pass
    return question_id